In [1]:
import os
import joblib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pytorch_lightning as pl
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error as mae
import seaborn as sns
import timm
from timm.optim import AdamW, Nadam
from timm.optim.madgrad import MADGRAD
from timm.scheduler.tanh_lr import TanhLRScheduler
from timm.scheduler.cosine_lr import CosineLRScheduler
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import wandb

from sklearn.pipeline import make_pipeline, make_union
seed = 457860
pl.seed_everything(seed)

Global seed set to 457860


457860

In [2]:
from config import Fields, root_path, out_path
from src.utils import read_train_test_revealed, read_population, DataMatrix
from src.eval import smape
from src.last_value_baseline import anamoly_correction, get_splits

In [3]:
# 6 months data as input and target as output
class GoData(Dataset):
    def __init__(self, data, flag="train", labels=None, pop=None, medians=None):
        super().__init__()
        self.data = data.copy()
        self.data[np.isnan(self.data)] = 0
        self.flag = flag
        self.labels = labels
        self.m = 18
        self.n = data.shape[0] - 4 - L
        self.pop = pop
        self.medians = medians
    
    def __len__(self):
        return self.data.shape[1]
    
    def __getitem__(self, idx):
        if self.flag == "train":
            stidx = int(np.random.rand() * (self.n - self.m) + self.m)
            enidx = stidx + L
            cfip_data = self.data[stidx:enidx, idx]
            new_med = self.data[enidx-1, idx]
            if new_med == 0:
                new_med = 1
            cfip_data = cfip_data / new_med * self.medians[0, idx]
            target = self.data[enidx+1:enidx+4, idx]
            target /= new_med * self.medians[0, idx]
            # target = np.clip(target, -4, 4)
        else:
            cfip_data = self.data[-L:, idx]
            target = np.array([1, 1, 1])
            if self.labels is not None:
                target = self.labels[:, idx]
                
        return cfip_data.astype(np.float32).reshape(-1, 1), target.astype(np.float32), self.pop[:, idx], self.medians[:, idx]


class SmapeLoss(nn.Module):
    def __init__(self):
        super(SmapeLoss, self).__init__()

    def forward(self, output, target):
        loss = 2 * (output - target).abs() / (output.abs() + target.abs() + 1e-6) * 100
        return loss.mean()
    

class SeqModel(nn.Module):
    def __init__(self, in_f, out_f, d, p, k=16):
        super().__init__()
        # self.fc1 = nn.Linear(in_f, d)
        self.h1 = nn.LSTM(in_f+1, d, num_layers=1, batch_first=True, bias=True, bidirectional=False)
        # self.h2 = nn.Linear(1, d * L, bias=True)
        self.out = nn.Linear(d * L, 3, bias=True)
        # self.bnorm1 = nn.BatchNorm1d(in_f, momentum=0.1)
        #self.relu = nn.ReLU()
        self.sigmoid = nn.Tanh()
        # self.bnorm2 = nn.BatchNorm1d(d)
        self.drop = nn.Dropout(p)
        self.drop2 = nn.Dropout(p)
    
    def forward(self, x, pop):
        b, l, n = x.shape
        # x = self.bnorm1(x)
        # x = self.fc1(self.drop(x))
        #x = self.relu(x)
        x2 = torch.log1p(pop.unsqueeze(1).repeat(1, l, 1))/100
        x = torch.cat([x, x2], axis=-1)
        x, _ = self.h1(x)  # , axis=-1))
        # x2 = self.sigmoid(self.h2(torch.log1p(med)/10))
        # x = x + x * self.sigmoid(self.h1(x))
        #x = self.relu(x)
        
        x = x.reshape(b, -1).contiguous()
        #🤐x = x1 
        x = self.out(self.drop2(x))
        return x


class LitModel(pl.LightningModule):
    """PL Model"""
    def __init__(
        self,
        in_f=6,
        out_f=1,
        d=128,
        p=0.1,
        lr=0.003,
        wd=0.001,
        grad_avg=True,
        steps=[30],
        gamma=0.1,
        
    ):
        super().__init__()
        self.save_hyperparameters()
        self.model = SeqModel(in_f, out_f, d, p)
        self.criterion = torch.nn.SmoothL1Loss()
        self.metric = SmapeLoss()

    def forward(self, x, pop):
        return self.model(x, pop)

    def step(self, batch):
        x, y, pop, med = batch
        yhat = self.forward(x, pop)
        loss = self.criterion(yhat, y)
        return loss, yhat, y, pop, med

    def training_step(self, batch, batch_idx):
        loss, preds, y, _, _ = self.step(batch)
        loss_corr = self.criterion(y*1000, preds*1000)
        self.log("train/loss", loss_corr, on_step=True, on_epoch=True, prog_bar=True)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        loss, preds, y, pop, med = self.step(batch)
        loss_corr = self.metric( (y + 1) * med * 100 / pop, (preds + 1) * med * 100 / pop)

        self.log("val/loss", loss_corr, on_step=False, on_epoch=True, prog_bar=True)
        return {"loss": loss}

    def test_step(self, batch, batch_idx):
        loss, preds, y = self.step(batch)
        return {"loss": loss, "preds": preds}

    def test_epoch_end(self, outputs):
        preds = torch.cat([o["preds"] for o in outputs], 0).cpu().numpy()
        self.log("output", preds)

    def configure_optimizers(self):
        optimizer = MADGRAD(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.wd)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, gamma=self.hparams.gamma,
                                                         milestones=self.hparams.steps)
        # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
        #                                                 total_steps=self.trainer.estimated_stepping_batches,
        #                                                 max_lr=self.hparams.lr,
        #                                                 div_factor=1, pct_start=0.0,
        #                                                 final_div_factor=25)
        return [optimizer], [{"scheduler": scheduler, "interval": "epoch"}]

    # def lr_scheduler_step(self, scheduler, step, metric):
    #     scheduler.step(epoch=self.current_epoch)  #        scheduler.step(epoch=step)


In [4]:
L = 8
BATCH_SIZE = 512
NUM_WORKERS = 7
IN_F = 1
D = 128
P = 0.05
LR = 0.001
WD = 0.00001
GRAD_AVG = True
MAX_EPOCHS = 50
STEPS = [20, 40]
GAMMA = 0.2
version = 'v0'

F = Fields()

df = read_train_test_revealed()
df["year"] = df[F.date].dt.year
pop_df = read_population()
df = pd.merge(df, pop_df, on=[F.cfips, "year"], how="left")
assert df[F.pop].isnull().sum() == 0


In [5]:
def anamoly_correction(ar):
    arr = ar[:]
    diffs = np.diff(arr)
    med = np.median(np.abs(diffs))
    p25 = np.percentile(diffs, 20)
    p75 = np.percentile(diffs, 80)
    iqr = p75 - p25
    upper_limit = med + 3 * iqr
    anam = np.where(np.abs(diffs) > upper_limit)[0]
    if len(anam) > 0:
        max_idx = np.max(anam)
        arr[:max_idx+1] = arr[max_idx+1]
        # arr[:max_idx+1] = arr[:max_idx+1] + diffs[max_idx] * 0.99
    return arr


def get_fold_data(df, pop_df, fold_id):
    df_tr, df_vl = get_splits(df, fold_id, "test")
    dm = DataMatrix(F.date, F.cfips, key_cols=[F.active, F.md, F.pop])
    dm.fit(df_tr)

    for col in range(dm.arr.shape[1]):
        dm.arr[:, col, 0] = anamoly_correction(dm.arr[:, col, 0]) 

    pop_df = pop_df.loc[pop_df.year == 2022]
    idx = [dm.item2id[cf] for cf in pop_df.cfips if cf in dm.item2id]
    pop = pop_df[F.pop].values[idx].reshape(1, -1)
    
    medians = dm.arr[[-1], :, 0]
    medians[medians == 0] = 1
    
    norm_arr_tr = dm.arr[:, :, 0]/medians - 1
    
    vl_dates = df_vl[F.date].unique()
    arr_vl = np.zeros(shape=(len(vl_dates), norm_arr_tr.shape[1]), dtype=np.float32)
    for i, date in enumerate(vl_dates):
        tmp_df = df_vl.loc[df_vl[F.date] == date]
        cfips = tmp_df[F.cfips].values
        actives = tmp_df[F.active].values
        idx = [dm.item2id[cf] for cf in cfips]
        arr_vl[i, idx] = actives
    norm_arr_vl = arr_vl/medians - 1
    return norm_arr_tr, norm_arr_vl, pop, medians

In [ ]:
scores = []
for fid in range(9):
    norm_arr_tr, norm_arr_vl, pop, medians = get_fold_data(df, pop_df, fid)
    tr_ds = GoData(norm_arr_tr,"train", None, pop, medians)
    tr_dl = DataLoader(tr_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    vl_ds = GoData(norm_arr_tr,"eval", norm_arr_vl, pop, medians)
    vl_dl = DataLoader(vl_ds, batch_size=BATCH_SIZE*2, drop_last=False, shuffle=False, num_workers=NUM_WORKERS)
    
    model = LitModel(IN_F, 4, D, P, lr=LR, wd=WD, grad_avg=GRAD_AVG, steps=STEPS, gamma=GAMMA)

    callbacks = [pl.callbacks.ModelCheckpoint(monitor='val/loss', save_top_k=2, mode='min', dirpath=f'data/checkpoints/{D}_{seed}_{version}', auto_insert_metric_name=False,
                                             filename="epoch={epoch}-val_loss={val/loss:.4f}")]
    logger = pl.loggers.wandb.WandbLogger(project='godaddy', name=f'nn_{fid}_{version}', tags=[f'{L}', f'{D}', f'fold_{fid}'], id=wandb.util.generate_id())
    logger.log_hyperparams({'D': D, 'P': P, 'LR': LR, 'WD': WD, 'GRAD_AVG': GRAD_AVG, 'BATCH_SIZE': BATCH_SIZE})
    trainer = pl.Trainer(accelerator='gpu', devices=[0], max_epochs=MAX_EPOCHS, logger=logger, callbacks=callbacks, deterministic=True)
    trainer.fit(model, tr_dl, vl_dl)
    score = trainer.callbacks[-1].best_model_score
    scores.append(score)
    wandb.finish()
scores = np.array([sc.cpu().numpy() for sc in scores])

[D 230301 19:36:07 last_value_baseline:45] Fold: 0 ; Split Date: 2022-09-01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mohsinhkn. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/py38/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /home/godaddy/data/checkpoints/128_457860_v0 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params
-------------------------------------------
0 | model     | SeqModel     | 70.7 K
1 | criterion | SmoothL1Loss | 0     
2 | metric    | SmapeLoss    | 0     
-------------------------------------------
70.7 K    Trainable params
0         Non-trainable params
70.7 K    Total params
0.283     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/py38/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
# get scores for each fold and save to a dict
# get avg score across 8 folds
# with and without correction
# cosine learning rate

In [ ]:
trainer.optimizers

In [ ]:
#scores = np.array([sc.cpu().numpy() for sc in scores])
scores.mean(), scores.std()

In [9]:
scores

array([2.531893 , 2.5159433, 2.6482525, 3.0653927, 2.5701752, 2.5922713,
       2.5454526, 2.6140354, 2.5255969], dtype=float32)